In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle

In [7]:
df=pd.read_csv('personality_datasert.csv')
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [10]:
df.isnull().sum()

Time_spent_Alone             0
Stage_fear                   0
Social_event_attendance      0
Going_outside                0
Drained_after_socializing    0
Friends_circle_size          0
Post_frequency               0
Personality                  0
dtype: int64

In [51]:
df.dtypes

Time_spent_Alone             float64
Stage_fear                     int64
Social_event_attendance      float64
Going_outside                float64
Drained_after_socializing      int64
Friends_circle_size          float64
Post_frequency               float64
Personality                    int64
dtype: object

In [13]:
## apply label encoder to Stage_fear
label_encoder=LabelEncoder()
df['Stage_fear']=label_encoder.fit_transform(df['Stage_fear'])


## apply label encoder to Drained_after_socializing
label_encoder=LabelEncoder()
df['Drained_after_socializing']=label_encoder.fit_transform(df['Drained_after_socializing'])


## apply label encoder to Personality
label_encoder=LabelEncoder()
df['Personality']=label_encoder.fit_transform(df['Personality'])

In [47]:
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,0,4.0,6.0,0,13.0,5.0,0
1,9.0,1,0.0,0.0,1,0.0,3.0,1
2,9.0,1,1.0,2.0,1,5.0,2.0,1
3,0.0,0,6.0,7.0,0,14.0,8.0,0
4,3.0,0,9.0,4.0,0,8.0,5.0,0


In [18]:
X=df.drop('Personality',axis=1)
y=df['Personality']

In [19]:
## Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)


In [21]:
from sklearn.preprocessing import StandardScaler
standardscaler=StandardScaler()
X_train_scaled=standardscaler.fit_transform(X_train)
X_test_scaled=standardscaler.transform(X_test)

In [50]:
## Save encoder And Scaler as Pickle file
with open('label_encoder_stage_fear.pkl','wb') as file:
    pickle.dump(label_encoder_stage_fear,file)

with open('label_encoder_drained_after_socializing.pkl','wb') as file:
    pickle.dump(label_encoder_drained_after_socializing,file)
    
  
with open('standardscaler.pkl','wb') as file:
    pickle.dump(standardscaler,file)

# ANN Implementation

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [29]:
# always input data in model like this
(X_train.shape[1],)

(7,)

In [31]:
# Build our ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

c:\Users\being\OneDrive\Desktop\ANN-Project-classification\.venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
import tensorflow
optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [38]:
## COmpile the model
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [44]:
# Setup earlystopping
early_stop = EarlyStopping(
    monitor='val_loss',     # you can also use 'val_accuracy'
    patience=10,             # number of epochs with no improvement before stopping
    restore_best_weights=True  # restores best weights to avoid overfitting
)

In [45]:
# Fit the model with early stopping
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9342 - loss: 0.2184 - val_accuracy: 0.9293 - val_loss: 0.2438
Epoch 2/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9385 - loss: 0.1981 - val_accuracy: 0.9276 - val_loss: 0.2437
Epoch 3/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9378 - loss: 0.2050 - val_accuracy: 0.9293 - val_loss: 0.2616
Epoch 4/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9374 - loss: 0.2052 - val_accuracy: 0.9293 - val_loss: 0.2386
Epoch 5/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9301 - loss: 0.2242 - val_accuracy: 0.9293 - val_loss: 0.2524
Epoch 6/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9378 - loss: 0.1943 - val_accuracy: 0.9293 - val_loss: 0.2586
Epoch 7/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9391 - loss: 0.1953 - val_accuracy: 0.9293 - val_loss: 0.2531
Epoch 8/100
73/73 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9344 - loss: 0.2159 - val_accuracy: 0.9293 - v

In [46]:
## Save Model

model.save('model.h5')

In [48]:
#####################################################################3

In [49]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("model.h5")

# Predict on test set
y_pred_prob = model.predict(X_test)

# If binary classification (sigmoid output)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Additional metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Accuracy: 0.9293

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       302
           1       0.92      0.94      0.93       278

    accuracy                           0.93       580
   macro avg       0.93      0.93      0.93       580
weighted avg       0.93      0.93      0.93       580

Confusion Matrix:
[[278  24]
 [ 17 261]]
